In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from functools import reduce
import datetime
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
def ftrans(num):
    num = int(num)
    if num == 0:
        return 0
    if num == 1000:
        return 1
    if num == 1500:
        return 2
    if num == 2000:
        return 3

def trans(num):
    num = int(num)
    if num == 0:
        return 0
    if num == 1:
        return 1000
    if num == 2:
        return 1500
    if num == 3:
        return 2000

In [141]:
where = pd.read_csv("card_where_amount_top.csv",encoding="GB18030")#33
consume = pd.read_csv("card_consume_amount_top.csv",encoding="GB18030")#29
time = pd.read_csv("card_time_amount_top.csv",encoding="GB18030")#32
time_DayofMonth = pd.read_csv("card_time_DayofMonth_amount_top.csv",encoding="GB18030")#13
time_DayofWeek = pd.read_csv("card_time_DayofWeek_amount_top.csv",encoding="GB18030")#11
time_HourofDay = pd.read_csv("card_time_HourofDay_amount_top.csv",encoding="GB18030")#13
time_MonthofYeah = pd.read_csv("card_time_MonthofYeah_amount_top.csv",encoding="GB18030")#31
time_DayofYeah = pd.read_csv("card_time_DayofYeah_amount_top.csv",encoding="GB18030")#13
print where.shape,time.shape,consume.shape,time_DayofMonth.shape,time_DayofWeek.shape,time_HourofDay.shape,time_MonthofYeah.shape,time_DayofYeah.shape

(21605, 33) (21631, 32) (21631, 29) (21631, 13) (21631, 11) (21631, 13) (21631, 31) (21631, 13)


In [11]:
def topk(k1,k2,k3,k4,k5,k6,k7,k8):
    where = pd.read_csv("card_where_amount_top.csv",encoding="GB18030").iloc[:,:k1]
    consume = pd.read_csv("card_consume_amount_top.csv",encoding="GB18030").iloc[:,:k2]
    time = pd.read_csv("card_time_amount_top.csv",encoding="GB18030").iloc[:,:k3]
    time_DayofMonth = pd.read_csv("card_time_DayofMonth_amount_top.csv",encoding="GB18030").iloc[:,:k4]
    time_DayofWeek = pd.read_csv("card_time_DayofWeek_amount_top.csv",encoding="GB18030").iloc[:,:k5]
    time_HourofDay = pd.read_csv("card_time_HourofDay_amount_top.csv",encoding="GB18030").iloc[:,:k6]
    time_MonthofYeah = pd.read_csv("card_time_MonthofYeah_amount_top.csv",encoding="GB18030").iloc[:,:k7]
    time_DayofYeah = pd.read_csv("card_time_DayofYeah_amount_top.csv",encoding="GB18030").iloc[:,:k8]
    return where,time,consume,time_DayofMonth,time_DayofWeek,time_HourofDay,time_MonthofYeah,time_DayofYeah

In [12]:
score_train = pd.read_table('data/train/score_train.txt',encoding="GB18030",sep=',',header=-1)
score_train.columns = ['id','college','score']
score_test = pd.read_table('data/test/score_final_test.txt',encoding="GB18030",sep=',',header=-1)
score_test.columns = ['id','college','score']
score_train_test = pd.concat([score_train,score_test])

In [13]:
college_train_test = score_train_test.groupby(["id","college"])["score"].max().unstack(["college"]).fillna(0)
college_name_list = list(college_train_test.columns)
college_train_test = college_train_test.rename(columns=dict(zip(college_name_list,["college_"+str(i) for i in college_name_list]))).reset_index()

In [14]:
college = pd.DataFrame(score_train_test.groupby(['college'])['score'].max()).reset_index()
college.columns = ['college','college_num']
score_train_test = pd.merge(score_train_test, college, how='left',on='college')
score_train_test['college_orderate'] = score_train_test['score']/score_train_test['college_num']
del score_train_test["college"]
#del score_train_test["score"]
score_train_test = pd.merge(score_train_test,college_train_test,how="left",on="id")

In [8]:
card_train = pd.read_table('data/train/card_train.txt',sep=',',header=-1)
card_train.columns = ['id','consume','where','how','time','amount','remainder']
card_test = pd.read_table('data/test/card_final_test.txt',sep=',',header=-1)
card_test.columns = ['id','consume','where','how','time','amount','remainder']
card_train_test = pd.concat([card_train,card_test])

In [9]:
card = pd.DataFrame(card_train_test.groupby(['id'])["time"].count()).reset_index().rename(columns={"time":"cardActionNum"})
card['remainderavg'] = card_train_test.groupby(['id'])['remainder'].mean()
card['remaindermax'] = card_train_test.groupby(['id'])['remainder'].max()
card['remaindermin'] = card_train_test.groupby(['id'])['remainder'].min()

In [15]:
def singleFeatureDeal(str1,feature):
    x_count = card_train_test.groupby(["id",str1])[feature].count().unstack([str1])
    x_count = x_count.rename(columns=dict(zip(x_count,[str(i)+"_count" for i in x_count])))

    x_sum = card_train_test.groupby(["id",str1])[feature].sum().unstack([str1])
    x_sum = x_sum.rename(columns=dict(zip(x_sum,[str(i)+"_sum" for i in x_sum])))

    x_mean = card_train_test.groupby(["id",str1])[feature].mean().unstack([str1])
    x_mean = x_mean.rename(columns=dict(zip(x_mean,[str(i)+"_mean" for i in x_mean])))

    x_max = card_train_test.groupby(["id",str1])[feature].max().unstack([str1])
    x_max = x_max.rename(columns=dict(zip(x_max,[str(i)+"_max" for i in x_max])))

    x_min = card_train_test.groupby(["id",str1])[feature].min().unstack([str1])
    x_min = x_min.rename(columns=dict(zip(x_min,[str(i)+"_min" for i in x_min])))

    x_median = card_train_test.groupby(["id",str1])[feature].median().unstack([str1])
    x_median = x_median.rename(columns=dict(zip(x_median,[str(i)+"_median" for i in x_median])))
    
    x = pd.concat([x_count,x_sum,x_mean,x_max,x_min,x_median],axis=1).reset_index().fillna(0)
    return x

In [16]:
card_how_amount = singleFeatureDeal("how","amount")

In [19]:
def Pipeline(seed,num,k1,k2,k3,k4,k5,k6,k7,k8,dropList1,dropList2):
      
    # train_test
    train = pd.read_table('data/train/subsidy_train.txt',encoding="GB18030",sep=',',header=-1)
    train.columns = ['id','money']
    test = pd.read_table('data/test/subsidy_final_test.txt',encoding="GB18030",sep=',',header=-1)
    test.columns = ['id']
    test['money'] = np.nan
    train_test = pd.concat([train,test])
    
    where,time,consume,time_DayofMonth,time_DayofWeek,time_HourofDay,time_MonthofYeah,time_DayofYeah = topk(k1,k2,k3,k4,k5,k6,k7,k8)
    print where.shape,time.shape,consume.shape,time_DayofMonth.shape,time_DayofWeek.shape,time_HourofDay.shape,time_MonthofYeah.shape,time_DayofYeah.shape
    train_test = pd.merge(train_test,score_train_test,how='left',on='id')
    train_test = pd.merge(train_test,card,how='left',on='id')
    train_test = pd.merge(train_test,card_how_amount,how='left',on='id')
    
    train_test = pd.merge(train_test,where,how='left',on='id')
    train_test = pd.merge(train_test,consume,how='left',on='id')
    train_test = pd.merge(train_test,time,how='left',on='id')
    train_test = pd.merge(train_test,time_DayofMonth,how='left',on='id')
    train_test = pd.merge(train_test,time_DayofWeek,how='left',on='id')
    train_test = pd.merge(train_test,time_HourofDay,how='left',on='id')
    train_test = pd.merge(train_test,time_MonthofYeah,how='left',on='id')
    train_test = pd.merge(train_test,time_DayofYeah,how='left',on='id')
    
    
    train_test.drop(dropList1,axis=1, inplace=True)
    train_test.drop(dropList2,axis=1, inplace=True)   
    
    train = train_test.loc[train_test.money.notnull(),:]
    test = train_test.loc[train_test.money.isnull(),:]
    
    #### Oversample
    del test["money"]
    Oversampling1000 = train.loc[train.money == 1000]
    Oversampling1500 = train.loc[train.money == 1500]
    Oversampling2000 = train.loc[train.money == 2000]
    for i in range(8):
        train = train.append(Oversampling1000)
    for j in range(11):
        train = train.append(Oversampling1500)
    for k in range(12):
        train = train.append(Oversampling2000)

    # model
    train = train.set_index(["id"]).fillna(-1)
    test = test.set_index(["id"]).fillna(-1)
    train['money'] = train['money'].apply(lambda x: ftrans(x))
    
    
    np.random.seed(seed)
    X_train = train.iloc[np.random.permutation(len(train))]
    y_train = X_train.money
    X_train = X_train.drop(['money'],axis=1)
    
    xgb_train   = xgb.DMatrix(X_train,y_train)
    xgb_test = xgb.DMatrix(test)
    
    xgb_params = {
        'booster': 'gbtree',
        #'objective': 'multi:softmax',
        'objective': 'multi:softprob',
        'num_class': 4,
        'max_depth': 5,
        'learning_rate': 0.04,
        'colsample_bytree': 0.8,
        'subsample': 0.8,
        'eval_metric': 'merror',
        'silent': 1,
    }
    
    watchlist = [(xgb_train,'train')]
    bst = xgb.train(xgb_params,xgb_train,100,watchlist,verbose_eval=False)
    yprob = bst.predict(xgb_test).reshape( test.shape[0], 4 )
    ylabel = np.argmax(yprob, axis=1)
    pd.DataFrame(yprob).to_csv("xgb_prob_model1_seed{}_{}.csv".format(seed,num))
    
    pred = pd.DataFrame({'studentid': test.reset_index()['id'],
                             'subsidy': ylabel})
    pred['subsidy'] = pred['subsidy'].apply(lambda x: trans(x))
    pred.to_csv("./xgb_pred_seed{}_{}.csv".format(seed,num), index=False)
    #print "数据量:", train.shape[0], ":", pred.shape[0]
    #print "有奖数量:", sum(train['money'] != 0), ":", sum(pred['subsidy'] != 0)
    print "1000奖学金数量：", sum(train['money'] == 1000), sum(pred['subsidy'] == 1000)
    print "1500奖学金数量：", sum(train['money'] == 1500), sum(pred['subsidy'] == 1500)
    print "2000奖学金数量：", sum(train['money'] == 2000), sum(pred['subsidy'] == 2000)
 
    pd.Series(bst.get_fscore()).reset_index().sort_values(0).to_csv("xgb_feature_seed{}_{}.csv".format(seed,num),encoding="GB18030")

In [20]:

dropList1 = ["校医院_count","其他_sum","其他_min","校医院_min","校医院_max","校医院_median","文印中心_median","洗衣房_min",
           "开水_min","college_13","教务处_max","其他_sum","其他_median","其他_max","college_1","college_12","college_18"]
#dropList1 =[]
dropList2 = []
Pipeline(100,1,15,15,15,15,15,15,15,15,dropList1,dropList2)

(21778, 15) (21806, 15) (21806, 15) (21806, 13) (21806, 11) (21806, 13) (21806, 15) (21806, 13)
1000奖学金数量： 0 2326
1500奖学金数量： 0 696
2000奖学金数量： 0 288


In [21]:

dropList1 = ["校医院_count","其他_sum","其他_min","校医院_min","校医院_max","校医院_median","文印中心_median","洗衣房_min",
           "开水_min","college_13","教务处_max","其他_sum","其他_median","其他_max","college_1","college_12","college_18"]
#dropList1 =[]
dropList2 = []
Pipeline(4000,2,15,15,15,15,15,15,15,15,dropList1,dropList2)

(21778, 15) (21806, 15) (21806, 15) (21806, 13) (21806, 11) (21806, 13) (21806, 15) (21806, 13)
1000奖学金数量： 0 2355
1500奖学金数量： 0 676
2000奖学金数量： 0 275
